In [3]:
import pandas as pd
import numpy as np
import requests
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,explained_variance_score

In [6]:
train_df = pd.read_csv("input/train.csv")
train_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.12,Premium,H,SI1,61.6,59.0,6.67,6.63,4.10,5363
1,1,1.14,Very Good,E,SI2,60.0,54.0,6.74,6.97,4.11,5593
2,2,0.90,Very Good,D,SI2,60.3,63.0,6.12,6.22,3.72,3534
3,3,0.71,Ideal,E,VS1,61.9,54.0,5.74,5.76,3.56,3212
4,4,0.34,Very Good,F,SI2,60.0,62.0,4.51,4.55,2.72,447


In [8]:
train_dummies=pd.get_dummies(train_df)
train_dummies.head()

,id,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0,1.12,61.6,59.0,6.67,6.63,4.10,5363,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1.14,60.0,54.0,6.74,6.97,4.11,5593,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,0.90,60.3,63.0,6.12,6.22,3.72,3534,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3,0.71,61.9,54.0,5.74,5.76,3.56,3212,0,0,...,0,0,0,0,0,0,1,0,0,0
4,4,0.34,60.0,62.0,4.51,4.55,2.72,447,0,0,...,0,0,0,0,0,1,0,0,0,0


In [44]:
test_df =train_dummies.drop(test_df.columns[[0]], axis='columns')
test_df

,depth,table,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,61.6,59.0,6.67,6.63,4.10,5363,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,60.0,54.0,6.74,6.97,4.11,5593,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,60.3,63.0,6.12,6.22,3.72,3534,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,61.9,54.0,5.74,5.76,3.56,3212,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,60.0,62.0,4.51,4.55,2.72,447,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,63.8,58.0,6.13,6.09,3.90,4120,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
40451,62.0,54.0,4.56,4.57,2.83,1266,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
40452,58.4,55.0,6.34,6.39,3.72,3269,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
40453,63.6,56.0,6.63,6.68,4.23,5893,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [35]:
columnas = [a for a in train_dummies.columns if a not in ["price"]]
X = train_dummies[columnas]
y = train_dummies["price"]

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(32364, 26) (8091, 26) (32364,) (8091,)


In [12]:
model = RandomForestRegressor(n_estimators=500,max_features="auto",max_depth=35,n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [13]:
print("R2_score",r2_score(y_test, y_pred))
print("Mean squared error",mean_squared_error(y_test, y_pred)**.5)
print("Mean absolute error",mean_absolute_error(y_test, y_pred))
print("explained variance score",explained_variance_score(y_test, y_pred))

R2_score 0.9801948523523516
Mean squared error 566.9235594618074
Mean absolute error 282.2009649796102
explained variance score 0.9801948545815392


In [14]:
scores = cross_val_score(model,X,y, cv=10)
print(np.mean(scores))

0.9799150192188092


In [52]:
model.fit(X, y)

RandomForestRegressor(max_depth=35, max_features=7, n_estimators=400)

In [54]:
test_df = pd.read_csv('input/predict.csv',index_col=0)
test_df.head()

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,0.33,Very Good,I,IF,62.0,58.0,4.44,4.46,2.76
1,1.21,Very Good,D,SI2,62.4,58.0,6.77,6.83,4.24
2,1.06,Very Good,D,SI1,59.3,60.0,6.64,6.71,3.96
3,0.36,Ideal,E,VVS1,61.4,57.0,4.64,4.61,2.54
4,0.70,Ideal,E,VS1,62.3,54.0,5.67,5.72,3.55


In [55]:
test_dummies=pd.get_dummies(test_df)
test_dummies.head()

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
id,,,,,,,,,,,,,,,,,,,,,
0,0.33,62.0,58.0,4.44,4.46,2.76,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,1.21,62.4,58.0,6.77,6.83,4.24,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1.06,59.3,60.0,6.64,6.71,3.96,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.36,61.4,57.0,4.64,4.61,2.54,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0.70,62.3,54.0,5.67,5.72,3.55,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [56]:
train_dummies.head()

,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.12,61.6,59.0,6.67,6.63,4.10,5363,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1.14,60.0,54.0,6.74,6.97,4.11,5593,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0.90,60.3,63.0,6.12,6.22,3.72,3534,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0.71,61.9,54.0,5.74,5.76,3.56,3212,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,0.34,60.0,62.0,4.51,4.55,2.72,447,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [60]:
model_test=model.predict(test_dummies)

In [63]:
RandomForest_test_2=pd.DataFrame(model_test).reset_index().rename(columns={"index":"id",0:"price"})

In [64]:
RandomForest_test_2.to_csv("output/RandomForest_test_2.csv",index=False)